# Custom Data Generation for YOLOv11 with VOCDataset and Training

1. Take pictures
1. Prepare environment
1. Prepare/Organize object images
3. Download VOCDataset for backing of object images
1. Overlay objects on backing images and generate YOLO labels
9000. Train model
1. Upload to Luxonis to convert to OpenVINO format

--------

## 1. Take Pictures

When taking pictures of objects:
- TAKE MANY PHOTOS AT ALL POSSIBLE ANGLES (15-ish photos, depending on how many angles we need)
- Good lighting and quality, of course
- Try to have the object be the only thing in frame (no extra objects that could possibly take away focus from the image)
- The object should be completely in frame and not cut off
- When you get all these images, place them in `~/MonsterVision5/uneditedObjects/` (you may have to manually create the folder if it does not exist)

Examples of some image angles/variation:

<img src="markdownimages/uneditedimage.png" alt="goodexample-baseimage" width="900" height="300">

For testing purposes, there are some sample images in the `sample-images/` folder with algaes, corals, notes, and carrots, you can duplicate the contents (but do not remove them!)

--------

## 2. Prepare Environment


### IMPORTANT THINGS TO NOTE WHEN RUNNING THIS BLOCK:
1. this cell block must be run first before anything else because it creates our venv!!!
1. If you don't have a kernel already selected, select `python version 3.10` (WE NEED THIS VERSION OF PYTHON)

Note on linux, you may need to install python3.10 as follows before running this block:

```bash
# 1. Update your package list and install the prerequisite for adding PPAs
sudo apt update
sudo apt install software-properties-common -y

# 2. Add the deadsnakes PPA (Press Enter when prompted)
sudo add-apt-repository ppa:deadsnakes/ppa

# 3. Update the package list again to include the new PPA repositories
sudo apt update

# 4. Install Python 3.10 and the venv module
sudo apt install python3.10 python3.10-venv -y
```

<img src="markdownimages/image.png" alt="goodexample-baseimage" width="550" height="200">


In [ ]:
import platform
print(platform.system())
import shutil
import sys
isWindows = False

if platform.system() == "Windows":
    # if machine is windows
    isWindows = True
    print("the next line should print 3.10, otherwise you are using the wrong version!!!!")
    !python --version
    !python -m venv .venv
    !.\.venv\Scripts\activate
    print("windows user!")
else:
    # if machine is NOT!!!! windows
    isWindows = False
    print ("mac/linux user")

    # Check if python3.10 exists in the system PATH
    if shutil.which("python3.10") is None:
        print("ERROR: python3.10 is not installed or not in PATH.")
        print("Please install python3.10 before running this script.")
        sys.exit(1)  # Terminate the script with an error code
    
    !python3.10 --version
    !python3.10 -m venv .venv
    !source .venv/bin/activate

Darwin
mac/linux user
Python 3.10.18


after running the code block, you will now have a venv! change the kernel again to python VENV 3.10, and only then can you run this next block

this is also something that MUST be run before running the subsequent code blocks, but then again youre supposed to be running all these in order either way :)

<img src="markdownimages/image2.png" alt="goodexample-baseimage" width="900" height="300">

#### Press "Install" if it asks to install `ipykernel`!!! ^^^

Check CUDA version if you have CUDA for pytorch installation and install dependencies for yolov11 and all other ultralytics yolo versions:

For pytorch: 
**Visit [pytorch.org/get-started/locally/](https://pytorch.org/get-started/locally/) to install the correct version of pytorch** (only change the url part of the command and if you have a newer version of CUDA than pytorch has then install the latest pytorch version). This could take a while depending on your network speed.

In [1]:
import os
absoluteMVpath = os.getcwd()
os.chdir(absoluteMVpath)
import platform
print(platform.system())
isWindows = False

if platform.system() == "Windows":
    # if machine is windows
    isWindows = True
    print("windows user!")
else:
    # if machine is NOT!!!! windows
    isWindows = False
    print ("mac/linux user")

!git clone --progress --verbose https://github.com/ultralytics/ultralytics

# we do not cd into ultralytics yet since we complete other object image tasks first
# operations are inside ultralytics
# create other folders that we will need later:

# -p checks if the directory already exists before making it

if isWindows:
    isWindows = True
    !mkdir -p .\ultralytics\objectImages
    !mkdir -p .\ultralytics\ultralytics\Dataset\images\test
    !mkdir -p .\ultralytics\ultralytics\Dataset\images\train
    !mkdir -p .\ultralytics\ultralytics\Dataset\labels\test
    !mkdir -p .\ultralytics\ultralytics\Dataset\labels\train
else:
    isWindows = False
    !mkdir -p ./ultralytics/objectImages
    !mkdir -p ./ultralytics/ultralytics/Dataset/images/test
    !mkdir -p ./ultralytics/ultralytics/Dataset/images/train
    !mkdir -p ./ultralytics/ultralytics/Dataset/labels/test
    !mkdir -p ./ultralytics/ultralytics/Dataset/labels/train
    
os.chdir(absoluteMVpath)
import subprocess
isCuda = False

# CHANGE URL BELOW:
try:
    subprocess.run(["nvidia-smi"], capture_output=True, text=True)
    # if machine is cuda
    isCuda = True
    %pip install "ultralytics" "tqdm>=4.41.0" "pillow" "rembg==2.0.28" "onnx" "blobconverter"
    %pip install torch torchvision --index-url https://download.pytorch.org/whl/cu126
except:
    # if machine is non-cuda
    isCuda = False
    %pip install "ultralytics" "tqdm>=4.41.0" "pillow" "rembg==2.0.28" "onnx" "blobconverter"
    %pip install torch torchvision --index-url https://download.pytorch.org/whl/cpu

print("\n\n\ncuda machine:", isCuda)

Darwin
mac/linux user
Cloning into 'ultralytics'...
POST git-upload-pack (182 bytes)
POST git-upload-pack (gzip 42559 to 21449 bytes)
remote: Enumerating objects: 77863, done.
remote: Counting objects: 100% (639/639), done.
remote: Compressing objects: 100% (305/305), done.
remote: Total 77863 (delta 535), reused 334 (delta 334), pack-reused 77224 (from 5)
Receiving objects: 100% (77863/77863), 42.26 MiB | 45.17 MiB/s, done.
Resolving deltas: 100% (58395/58395), done.
  Using cached ultralytics-8.3.241-py3-none-any.whl.metadata (37 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached pillow-12.0.0-cp310-cp310-macosx_11_0_arm64.whl.metadata (8.8 kB)
  Using cached rembg-2.0.28-py3-none-any.whl.metadata (9.6 kB)
  Using cached onnx-1.20.0-cp310-cp310-macosx_12_0_universal2.whl.metadata (8.4 kB)
  Using cached blobconverter-1.4.3-py3-none-any.whl.metadata (7.8 kB)
  Using cached aiohttp-3.8.1-cp310-cp310-macosx_11_0_arm64.whl.metadata (7.3 kB)
  Using cached asy

## 3. Prepare Object Images

Next we will clean up the data (images) we have by organizing/editing them

Example: (note how there is no background and it is cropped right to the edge of the object)

ORIGINAL:
<img src="markdownimages/algae_6411.jpeg" alt="unedited" width="300" height="390">
EDITED:
<img src="markdownimages/editedimage.png" alt="goodexample-editedimage" width="300" height="300">

---------

#### BEFORE RUNNING THIS CODE CELL: Organize your images in folders within the original folder 
- Within `uneditedObjects`, make folders with the names of each gamepiece you are using and place each image in its respective directory.
- **EVEN IF YOU HAVE ONLY ONE TYPE OF GAMEPIECE**, YOU MUST STILL PUT IT IN A FOLDER WITH ITS NAME WITHIN uneditedObjects!
- example: if your object images are `algae` and `coral`, your directory should look something like this:
```
MonsterVision5
├── uneditedObjects
|     ├── algae
|     |     └── images of algaes go here
|     └── coral
|           └── images of corals go here
|
└───── other folders we dont care about right now
```


In [2]:
import json
from PIL import Image
os.chdir(absoluteMVpath)

if isWindows:
    print("is windows")
    classNames = (os.listdir('.\\uneditedObjects\\')).sort()
    CLASSES = {key: index for index, key in enumerate(classNames)} # DON'T EDIT
    os.environ['CLASSES'] = json.dumps(CLASSES)
    print(json.dumps(CLASSES))

    print("renaming images...")
    # rename all images to fit the naming conventions
    for object in os.listdir('.\\uneditedObjects\\'):
        number = 0
        for image in os.listdir('.\\uneditedObjects\\' + object):
            os.rename('.\\uneditedObjects\\' + object + '\\' + image, '.\\uneditedObjects\\' + object + '_' + str(number) + '.png')
            number += 1

    # we first use rembg to remove background of images from our base images
    # THEN we put them through Pillow to crop the images using the detected removed background
    # then it saves it in ultralytics

    print("removing background...")
    !mkdir .\temporary
    !rembg p .\uneditedObjects temporary

    print("cropping images...")
    for i in os.listdir('.\\temporary\\'):
        img = Image.open(f".\\temporary\\{i}").convert("RGBA") # MUST BE PNG FILE TO USE RGBA >:/
        alpha = img.split()[3] # alpha channel is apparently the 4th channel in RGBA, it means transparency
        # threshold makes sure that even if there are translucent pixels in the background we crop them
        threshold = 10  # tweak this value if needed (0–255), 10 is pretty good for now
        alpha = alpha.point(lambda p: p > threshold and 255)
        # alpha.point is a function applied to all pixels in an image
        # the threshold checks how translucent it is and either makes it fully transparent or opaque
        img.putalpha(alpha) # recombine to an RGBA image
        bbox = alpha.getbbox() # find the bounding box of non-transparent pixels (find the edge of where there actually is an image object)

        if bbox:
            cropped = img.crop(bbox)
            cropped.save(f".\\ultralytics\\objectImages\\{i}") # the Final Destination (haha) of the images
            
    print("images are now in \\ultralytics\\objectImages\\!\nplease confirm that EVERY image has been edited correctly\n\n\n\n\n\n")
    !del .\temporary
    !del .\uneditedObjects
else:
    print("is NOT windows")
    classNames = os.listdir('./uneditedObjects/')
    CLASSES = {key: index for index, key in enumerate(classNames)} # DON'T EDIT
    os.environ['CLASSES'] = json.dumps(CLASSES)
    print(json.dumps(CLASSES))

    print("renaming images...")
    # rename all images to fit the naming conventions
    for object in os.listdir('./uneditedObjects/'):
        number = 0
        for image in os.listdir('./uneditedObjects/' + object):
            os.rename('./uneditedObjects/' + object + '/' + image, './uneditedObjects/' + object + '_' + str(number) + '.png')
            number += 1

    # we first use rembg to remove background of images from our base images
    # THEN we put them through Pillow to crop the images using the detected removed background
    # then it saves it in ultralytics

    print("removing background...")
    !mkdir ./temporary/
    !rembg p ./uneditedObjects temporary

    print("cropping images...")
    for i in os.listdir('./temporary/'):
        img = Image.open(f"./temporary/{i}").convert("RGBA") # MUST BE PNG FILE TO USE RGBA >:/
        alpha = img.split()[3] # alpha channel is apparently the 4th channel in RGBA, it means transparency
        # threshold makes sure that even if there are translucent pixels in the background we crop them
        threshold = 10  # tweak this value if needed (0–255), 10 is pretty good for now
        alpha = alpha.point(lambda p: p > threshold and 255)
        # alpha.point is a function applied to all pixels in an image
        # the threshold checks how translucent it is and either makes it fully transparent or opaque
        img.putalpha(alpha) # recombine to an RGBA image
        bbox = alpha.getbbox() # find the bounding box of non-transparent pixels (find the edge of where there actually is an image object)

        if bbox:
            cropped = img.crop(bbox)
            cropped.save(f"./ultralytics/objectImages/{i}") # the Final Destination (haha) of the images
            
    print("images are now in /ultralytics/objectImages/!\n\nplease confirm that EVERY image has been edited correctly\n\n")
    !rm -rf ./temporary/
    !rm -rf ./uneditedObjects/*

is NOT windows
{}
renaming images...
removing background...
0it [00:00, ?it/s]
cropping images...
images are now in /ultralytics/objectImages/!

please confirm that EVERY image has been edited correctly


zsh:1: no matches found: ./uneditedObjects/*


**before proceeding, please check the images to make sure they have been edited correctly (they are in `/ultralytics/objectImages/`)! If some images are not edited correctly, remove them or run through the process again**

--------

## 4. Download VOCDataset for Backing of Object Images
We use the VOCDataset or Visualized Object Classes Dataset which is a dataset that contains many images with labels for training of pascal. We are just extracting the images from a few of the datasets for backing images for our yolo training images.

Download the 2007 and 2012 VOCDataset and put them in a separate directory (may take 10+ minutes depending on wifi) and extract the downloaded .tar archive files (should create folders `VOC2007` and `VOC2012` in `VOCdevkit`. If not then run again):

In [ ]:
os.chdir(absoluteMVpath + "/ultralytics")
if isWindows:
    isWindows = True
    !mkdir -p .\VOCdevkit\test
    !mkdir -p .\VOCdevkit\train
    !curl -L -o .\VOCdevkit\pascal-voc-2012-dataset.zip https://www.kaggle.com/api/v1/datasets/download/gopalbhattrai/pascal-voc-2012-dataset
    !tar -xvf .\VOCdevkit\pascal-voc-2012-dataset.zip -C .\VOCdevkit\
    print("got VOCdevkit")
else:
    isWindows = False
    !mkdir -p ./VOCdevkit/test
    !mkdir -p ./VOCdevkit/train
    !curl -L -o ./VOCdevkit/pascal-voc-2012-dataset.zip https://www.kaggle.com/api/v1/datasets/download/gopalbhattrai/pascal-voc-2012-dataset
    !unzip ./VOCdevkit/pascal-voc-2012-dataset -d ./VOCdevkit/
    

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0--:-- --:--:--     0
  0 3605M    0 3599k    0     0  3628k      0  0:16:57 --:--:--  0:16:57 8089k
curl: (56) Failure writing output to destination, passed 16375 returned 0
Archive:  ./VOCdevkit/pascal-voc-2012-dataset
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of ./VOCdevkit/pascal-voc-2012-dataset or
        ./VOCdevkit/pascal-voc-2012-dataset.zip, and cannot find ./VOCdevkit/pascal-voc-2012-dataset.ZIP, period.


In [ ]:
os.chdir(absoluteMVpath + "/ultralytics")
!mkdir -p ./VOCdevkit 
!mkdir -p ./VOCdevkit/test
!mkdir -p ./VOCdevkit/train
# -p (lowercase) checks if the directory already exists before making
# the curl command is pre-installed on Mac
!curl -L -o ./pascal-voc-2012-dataset https://www.kaggle.com/api/v1/datasets/download/gopalbhattrai/pascal-voc-2012-dataset

!unzip ./VOCdevkit/pascal-voc-2012-dataset -d ./VOCdevkit/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 27 3605M   27  986M    0     0  42.7M      0  0:01:24  0:00:23  0:01:01 47.1M02:12  0:00:05  0:02:07 32.0M0  41.8M      0  0:01:26  0:00:18  0:01:08 46.8M
curl: (56) Failure writing output to destination, passed 16366 returned 135
Archive:  ./VOCdevkit/pascal-voc-2012-dataset
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of ./VOCdevkit/pascal-voc-2012-dataset or
        ./VOCdevkit/pascal-voc-2012-dataset.zip, and cannot find ./VOCdevkit/pascal-voc-2012-dataset.ZIP, period.


In [ ]:
os.chdir(absoluteMVpath + "/ultralytics/VOCdevkit")
for i in os.listdir('./VOC2012_test/VOC2012_test/JPEGImages'): # listdir wil return a list of strings of every file and folder name in the DIR
    os.rename('./VOC2012_test/VOC2012_test/JPEGImages/' + i, './test/' + i) # takes original path and appends each image file then moves it by "renaming" it
for i in os.listdir('./VOC2012_train_val/VOC2012_train_val/JPEGImages'):
    os.rename('./VOC2012_train_val/VOC2012_train_val/JPEGImages/' + i, './train/' + i)

os.chdir(absoluteMVpath + "/ultralytics/VOCdevkit")
if isWindows:
    isWindows = True
    !rmdir ./VOC2012_test/
    !rmdir ./VOC2012_train_val/
    !del pascal-voc-2012-dataset
else:
    isWindows = False
    !rm -rf ./VOC2012_test/
    !rm -rf ./VOC2012_train_val/
    !rm pascal-voc-2012-dataset

os.chdir(absoluteMVpath + "/ultralytics")

## 5. Overlay objects on backing images and generate YOLO labels

This script automatically generates synthetic training data for object detection models (such as YOLO) by compositing foreground object images onto random background images.

### Overview:
1. Randomly selects an object image (with transparent background) and a random background image.
2. Scales the object to a random percentage of the background’s size.
3. Pastes the scaled object onto a random position within the background.
4. Saves the resulting composite image to the dataset folder (e.g., ./ultralytics/Dataset/images/...).
5. Generates a corresponding YOLO-format label (.txt) containing the object’s class ID and bounding box coordinates
   (normalized x_center, y_center, width, height).
6. Designed for multiprocessing to efficiently generate large datasets.

### Functions:
- stackAndScaleImage(): Scales and pastes one image onto another using PIL.
- selectScaleAndCreateYoloLabels(): Combines object and background, computes YOLO label data.
- combineRandomImages(): Picks random images from directories and combines them.
- makeImage(): Creates and saves a single labeled synthetic training image.

In [ ]:
os.chdir(absoluteMVpath + "/ultralytics")

In [ ]:
%%writefile dataGen.py 
# we write to a new file so we can do multiprocessing on a separate python file (we cant do multiprocessing in .ipynb)
from PIL import Image
import json
import random
import os
from tqdm import tqdm as progressBar
import threading
import multiprocessing
import concurrent.futures
import argparse



def stackAndScaleImage(objectImage, backgroundImage, scalePercent, position): # Takes a first PNG image, scales it down a certain percentage and pastes it on a second PNG image.

# objectImage: PIL image of object
# backgroundImage: PIL image of background
# scalePercent: Percentage to scale down the first image. (MAKE FLOAT INSTEAD OF DUMB STUFF)
# position: Tuple of (x, y) coordinates to paste the scaled image.

  objectImage = Image.open(objectImage)
  backgroundImage = Image.open(backgroundImage)

  # Scale down the first image
  width, height = objectImage.size # extract the width and height of object
  newWidth = int(width * scalePercent) # create a new width for object based on the scalePercent and makes it an int
  newHeight = int(height * scalePercent) # create a new height for object based on the scalePercent and makes it an int
  objectScaled = objectImage.resize((newWidth, newHeight)) # use the resize method of a PIL Image to scale object to the new_width and new_height
 
  # Scaled the hight to add more variation to the data
  width, height = objectScaled.size # Reset the width and height variables to be the new width and height of object after scaling
  randomHeightScale = random.uniform(0.8, 1.2) # Choose a random scalePercent between the minimum and maximum values.
  # randomHeightScale = 1.0
  newHeight = int(height * randomHeightScale) # Calculate the newHeight based on the random scalePercent above
  if newHeight > backgroundImage.height:
    newHeight = backgroundImage.height
    print("triggered", newHeight)
  objectScaled = objectScaled.resize((width, newHeight)) # Resize the image just like above
  
  backgroundImage.paste(objectScaled, position, objectScaled) # Second parameter makes it so that the pixels with no value meant to be clear stay clear and aren't black

  return (backgroundImage, newHeight) # Return the final stacked image




# SCALE PERCENTAGE IS HOW MUCH OF THE FRAME YOU WANT TO TAKE UP NOT HOW MUCH YOU WANT TO SCALE THE OBJECT IMAGE DOWN
def selectScaleAndCreateYoloLabels(objectPath, backgroundPath, minSizePercent, maxSizePercent, objectClassStr, CLASSES): 
  # Combines two images by pasting a scaled version of an object onto a background. SCALES BASED ON WIDTH

  # objectPath: Path to the object image.
  # backgroundPath: Path to the background image.
  # minSizePercent: Minimum percentage to scale down object's WIDTH.
  # maxSizePercent: Maximum percentage to scale down object's WIDTH.

  object = Image.open(objectPath) # Open the first image as object
  background = Image.open(backgroundPath) # Open the second image as background

  objectWidth, objectHeight = object.size # Extract the width and height of object as objectWidth and objectHeight
  backgroundWidth, backgroundHeight = background.size # Extract the width and height of background as backgroundWidth and backgroundHeight

  # Choose a random scalePercent between the minimum and maximum values provided as parameters
  # NEVER ABOVE 1.0(?)
  scalePercent = random.uniform(minSizePercent, maxSizePercent)

  # as background's width before applying this random scale percent (should really be same as shortest side?)
  # baseScale = backgroundWidth / objectWidth # The baseScale is the ratio of how big background's width is compared to object's width (ONLY WITH MEASUREMENTS)
  if backgroundWidth <= backgroundHeight: # scale based on shortest side of background
    baseScale = backgroundWidth/objectWidth
    used = "width shortest so base scale is based on width"
  else:
    baseScale = backgroundHeight/objectHeight
    used = "height shortest so base scale is based on height"

  # print({
  #   "backgroundWidth": backgroundWidth,
  #   "object.width": object.width,
  #   "objectWidth": objectWidth,
  #   "baseScale": baseScale,
  #   "scalePercent": scalePercent,
  #   "baseScale * scalePercent": baseScale * scalePercent,
  #   "object.width * scalePercent/100": object.width * scalePercent/100,
  #   "object.height * scalePercent/100": object.height * scalePercent/100,
  # })


  scalePercent = baseScale * scalePercent # fix the scalePercent to include the baseScale between the 2 images and be proportional

  # Choose a random position for object in background
  widthOfObjectAfterScaling = int(object.width * scalePercent)   # predict width of object after scaling so you can choose a random width in bounds of background
  heightOfObjectAfterScaling = int(object.height * scalePercent) # predict height of object after scaling so you can choose a random height in bounds of background
  x = random.randint(0, backgroundWidth - widthOfObjectAfterScaling)     # select random x position for object in background

  # ERROR IS HERE: The error is because background isn't tall enough to fit object even after scaling so the randint is trying to selced from 0 to a negative number
  y = random.randint(0, backgroundHeight - heightOfObjectAfterScaling)   # select random y position for object in background

  (combinedImage, finalHeight) = stackAndScaleImage(objectPath, backgroundPath, scalePercent, (x, y)) # Use the stack_scaled_images function to combine the two images

  # finalWidth, finalHeight = combinedImage.size # WRONG

  # Save the paste_parameters as a json object
  debugParameters = {
    "width_background": backgroundWidth,
    "height_background": backgroundHeight,
    "paste_x": x,
    "paste_y": y,
    "paste_width": widthOfObjectAfterScaling,
    "paste_height": finalHeight,
    "scale_type": used
  }
  # The position for object to be pasted onto background is randomly selected within the bounds of background

  # https://docs.cogniflow.ai/en/article/how-to-create-a-dataset-for-object-detection-using-the-yolo-labeling-format-1tahk19/
  # YOLO labeling parameters
  pasteParametersYolo = {
    "objectClassNum": CLASSES[objectClassStr],
    "x_center": (x + widthOfObjectAfterScaling/2.0) / backgroundWidth, # calculate what percentage of the width of background the center of scaled object will be
    "y_center": (y + finalHeight/2.0) / backgroundHeight, # calculate what percentage of the height of background the center of scaled object will be
    "width": widthOfObjectAfterScaling / backgroundWidth, # calculate what percentage of the width of background does scaled object take
    "height": finalHeight / backgroundHeight, # calculate what percentage of the height of background does scaled object take
  }

  return (combinedImage, debugParameters, pasteParametersYolo) # return the PIL image object after stacking, the parameters used for pasting, and the parameters used for pasting in a YOLO suitable notation





def combineRandomImages(directory1, directory2, minSizePercent, maxSizePercent, CLASSES):
  # Combines two random images from the two directories using the first function, returns a PIL Image object with the combined images

  #   directory1: Path to the first directory.
  #   directory2: Path to the second directory.
  #   minSizePercent: Minimum percentage to scale down the first image.
  #   maxSizePercent: Maximum percentage to scale down the first image.

  # Get all files in directories. If one isn't an image it could break
  objectFiles = os.listdir(directory1) # Get a list of all files in the first directory
  backgroundFiles = os.listdir(directory2) # Get a list of all files in the second directory

  imageChosen = random.choice(objectFiles)
  objectClass = imageChosen.split("_")[0]

  # Choose random images
  objectPath = os.path.join(directory1, imageChosen) # Choose a random file from the first directory
  backgroundPath = os.path.join(directory2, random.choice(backgroundFiles)) # Choose a random file from the second directory

  # Use the combine_images_james_xy function to combine the two images.
  combinedImage, debugParameters, pasteParametersYolo = selectScaleAndCreateYoloLabels(objectPath, backgroundPath, minSizePercent, maxSizePercent, objectClass, CLASSES)

  # print(pasteParametersYolo)

  return (combinedImage, debugParameters, pasteParametersYolo) # re-return all of the returns from the combine_images_james_xy function

# def makeImage(testOrTrain="test", numImages=10, minSizePercent=.05, maxSizePercent=.8, i=-1):
def makeImage(args):
  (testOrTrain, numImages, minSizePercent, maxSizePercent, CLASSES, i) = args
  if i == -1:
    raise ValueError("Invalid i Value")
  # Combine the images from directory1 (game object) with images from directory2 (backgrounds).
  combinedImage, debugParameters, pasteParametersYolo = combineRandomImages("./objectImages", "./VOCdevkit/"+testOrTrain, minSizePercent, maxSizePercent, CLASSES)
  # Figure out a file name based on the current iteration and type of dataset
  baseFilename = f"{testOrTrain}_{i:0{6}d}" # Max 100000 file names
  # Save the image to the specified folder based on type of data set and use the above created filename
  combinedImage.save('./ultralytics/Dataset/images/'+testOrTrain+'/'+baseFilename+'.png')

  # Open/create a text file with the same name as the image and add the paste_parameters_yolo to it
  with open('./ultralytics/Dataset/labels/'+testOrTrain+'/'+baseFilename+'.txt', "w") as f:
    yoloData = pasteParametersYolo
    if round(yoloData['x_center'],6) > 1 or round(yoloData['y_center'],6) > 1:
      print("ERROR at", i)
      print(yoloData)
      print(debugParameters)
    f.write(f"{yoloData['objectClassNum']} {round(yoloData['x_center'],6)} {round(yoloData['y_center'],6)} {round(yoloData['width'],6)} {round(yoloData['height'],6)}") # write data and round it to the correct decimal places (first digit in string is the class)



# Main function that ties together all of the other functions to make it work
def makeData(testOrTrain="test", numImages=10, minSizePercent=.05, maxSizePercent=.8):
  args = tuple([(testOrTrain, numImages, minSizePercent, maxSizePercent, CLASSES, i) for i in range(numImages)]) # creates a tuple of tuple

  with concurrent.futures.ProcessPoolExecutor() as executor:
    executor.map(makeImage, args)
    print("yippie!!! it works") # to make sure it works :)

if __name__ == "__main__":
  parser = argparse.ArgumentParser()
  parser.add_argument("--classes", type=str)
  parser.add_argument("--testOrTrain", default="test", type=str)
  parser.add_argument("--numImages", default=2, type=int)
  parser.add_argument("--minSizePercent", default=.05, type=float)
  parser.add_argument("--maxSizePercent", default=.8, type=float)

  inputArgs = parser.parse_args()

  CLASSES = json.loads(inputArgs.classes.replace("'", '"'))
  testOrTrain = inputArgs.testOrTrain
  numImages = inputArgs.numImages
  minSizePercent = inputArgs.minSizePercent
  maxSizePercent = inputArgs.maxSizePercent

  testOrTrain = testOrTrain.lower()

  makeData(testOrTrain, numImages, minSizePercent, maxSizePercent)

Now we generate the test and train image sets. You should pick a size percent that makes sense for the application. Here are the lines to make a test and train dataset:

(This is going to take a few minutes on some machines)

In [ ]:
# train code
os.chdir(absoluteMVpath + "/ultralytics")
!python3 dataGen.py --classes="$CLASSES" --testOrTrain="train" --numImages=15000 --minSizePercent=.1 --maxSizePercent=.8

In [ ]:
# test code
os.chdir(absoluteMVpath + "/ultralytics")
!python3 dataGen.py --classes="$CLASSES" --testOrTrain="test" --numImages=7500 --minSizePercent=.1 --maxSizePercent=.9

Now we create the dataset config file for training yolo but **make sure to enumerate the classes at the bottom and pick names that are appropriate**:

(USE THE NAMES LISTED IN CLASSNAMES) AND PUT THEM IN THE SAME ORDER AS IN THE LIST

In [ ]:
print(classNames)
os.chdir(absoluteMVpath + "/ultralytics/ultralytics/cfg/datasets")

In [ ]:
%%writefile CUSTOM.yaml
# Ultralytics YOLO 🚀, AGPL-3.0 license
# Documentation: # Documentation: https://docs.ultralytics.com/datasets/detect/voc/
# Example usage: yolo train data=CUSTOM.yaml
# parent
# ├── ultralytics
# └── datasets
#     └── CUSTOM

# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
path: ./ultralytics/Dataset/
train: images/train # train images (relative to 'path')  16551 images
val: images/test # val images (relative to 'path')  4952 images
test: images/test # test images (optional)






# Set the number of classes
nc: 4 # EDIT TO REFLECT CORRECT AMOUNT OF CLASSES

# Classes
names: # EDIT HERE
  0: coral
  1: note
  2: algae
  3: carrot
  

## 6. Train Model

Now that we have the dataset we can train the model. First we will run the checks command to make sure everything is good (it has all the libraries, the computer is able to train it, etc).
We are going to use the nano version of yolov11 because it is the smallest, fastest, and most lightweight version. We will specify the name of the custom data config file that we created above as well as yolov11n.pt for the nano version of the pretrained model. I've decided on 10 epochs for just a short retraining of the model. The imgsz is 640 which will resize all of the images we created to 640x640 for input into the model. This number can be changed for speed over accuracy if needed. I set the batch size to use 90% of the available GPU memory as well as set cache to true to improve speed.

--------

my face when yolov12

In [ ]:
!yolo checks
import torch
print(torch.cuda.is_available())
!yolo help
!yolo settings datasets_dir=ultralytics
# i am a genuous....
from ultralytics import settings
print(settings)

imgsz is set to 512 because it must be a multiple of 32 and not greater than 530 to work with the camera with the OV... sensor

KEEP IN MIND:
if you didn't already know, training takes a while (as in I started training at around 1 am. it is now 8 am). Depending on what system you're on, it will probably take less long

In [ ]:
os.chdir(absoluteMVpath)
if isCuda:
    isCuda = True
    !yolo detect train data="./ultralytics/ultralytics/cfg/datasets/CUSTOM.yaml" model="yolo12n.pt" epochs=2 imgsz=512 name="lakeMONSTER" batch=0.5 patience=2 cache=disk workers=8 device=0 project=./datagenTraining/
else:
    isCuda = False
    !yolo detect train data="./ultralytics/ultralytics/cfg/datasets/CUSTOM.yaml" model="yolo12n.pt" epochs=2 imgsz=512 name="lakeMONSTER" batch=0.5 patience=2 cache=disk workers=8 device=cpu project=./datagenTraining/

## 7. Upload to Luxonis to convert to OpenVINO format
Run the below code to get your .blob to be used in


In [ ]:
import blobconverter

!yolo export model="best.pt" format=onnx opset=13 simplify

blobconverter.from_onnx(model="yolo12n.onnx", shaves=6, data_type="FP16", output_dir="./finalProduct/")


## End
You are done with this notebook! The next steps of using the models come in after you set up MonsterVision on the robot. Refer to the README.md for more clear instructions on when the models come in.